In [1]:
import pandas as pd
import re
import numpy as np
df_银行格式 = pd.DataFrame(columns = ['类罪类型',
'反洗钱报告编号',
'场景类型',
'交易ID',
'客户编号',
'交易日期',
'明细来源',
'业务种类',
'业务条线',
'报送业务条线',
'明细序号',
'定活存标志',
'账卡标志',
'客户账号',
'客户名称',
'对公对私标志',
'客户年龄',
'国际卡卡币种',
'币种代码',
'存款种类',
'可售产品编号',
'资金用途代码',
'客户类型代码',
'交易时间',
'交易金额',
'折人民币金额',
'折美元金额',
'借贷标志',
'收付标志',
'交易种类',
'交易方式',
'汇款种类',
'交易渠道种类',
'交易流水号',
'全局事件跟踪号',
'交易柜员号',
'交易代码',
'交易机构',
'交易机构名称',
'交易发生地',
'交易所在国家地区代码',
'交易方向',
'凭证号码',
'凭证种类',
'摘要代码',
'摘要描述',
'账户性质',
'账户类型',
'资金用途',
'开户机构',
'开户机构名称',
'开户机构行政区划',
'反洗钱机构',
'反洗钱机构名称',
'开户日期',
'销户日期',
'上次明细交易日',
'产品种类',
'交易对手系统账号',
'交易对手账号修饰符',
'交易对手账户类型',
'交易对手账户性质',
'交易对手对公对私标志',
'交易对手客户账号',
'交易对手客户编号',
'交易对手客户名称',
'交易对手本外行标识',
'交易对手证件号码',
'交易对手证件类型',
'交易对手证件类型说明',
'交易对手商户编号',
'交易对手商户种类',
'交易对手开户机构',
'交易对手开户机构行政区划',
'交易对手定活存标志',
'交易对手金融机构代码',
'交易对手金融机构名称',
'交易对手金融机构行政区划',
'交易对手金融机构代码网点类型',
'交易对手国籍地区',
'终端信息',
'大额豁免标志',
'老可疑豁免标志',
'新可疑豁免标志',
'代理人姓名',
'代理人证件类型',
'代理人证件类型说明',
'代理人证件号码',
'代理人电话',
'代理人国籍地区',
'是否被代办',
'是否跨境交易',
'结算类型现金转账消费',
'是否跨行交易',
'是否异地交易',
'补录机构',
'账户余额',
'备注1',
'备注2'])

Efile =  pd.ExcelFile(r'C:\Users\zqy\Documents\WeChat Files\wxid_u74gwfzs932921\FileStorage\File\2022-03\经侦数据demo\4555043320200821171052\账户交易明细表.xlsx')
sheet_list = pd.ExcelFile(r'C:\Users\zqy\Documents\WeChat Files\wxid_u74gwfzs932921\FileStorage\File\2022-03\经侦数据demo\4555043320200821171052\账户交易明细表.xlsx').sheet_names

all_流水完整 = pd.DataFrame()
for i in range(0,len(sheet_list)):
    df = Efile.parse(sheet_name = sheet_list[i])
    all_流水完整 = all_流水完整.append(df)
    print(i)
    if i ==1:
        break

all_流水完整 = all_流水完整.applymap(lambda x: re.sub('nan','',str(x)))

# all_流水完整['交易金额'] = pd.to_numeric(all_流水完整['交易金额'],errors='coerce')  #交易金额指定浮点型
# all_流水完整['交易余额'] = all_流水完整['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
# all_流水完整['交易余额'] = all_流水完整['交易余额'].map(lambda x: float(x))

print(all_流水完整.dtypes)

all_流水完整 = all_流水完整[all_流水完整['查询反馈结果']=='01']


df_代码 = pd.read_excel(r'K:\代码值.xlsx',dtype = 'object',sheet_name = '代码值（全）')
df_币种 = df_代码[df_代码['值域名称'] =='币种']
df_币种英文码 = df_代码[df_代码['值域名称'] =='币种英文码'][['代码编号','代码名称']]
df_币种英文码.columns = ['英文码','代码名称']
df_代码对应 = pd.merge(df_币种,df_币种英文码,how = 'left',on ='代码名称')
df_流水币种 = pd.merge(all_流水完整,df_代码对应,how = 'left',left_on = '交易币种',right_on = '英文码')
print(df_流水币种)


df_银行格式['客户编号'] = all_流水完整['交易账号']
df_银行格式['交易日期'] = all_流水完整['交易日期'].str[0:10]
df_银行格式['客户账号'] = all_流水完整['交易账卡号']
df_银行格式['客户名称'] = all_流水完整['交易户名']
df_银行格式['对公对私标志'] = all_流水完整['交易户名'].map(lambda x:'0' if len(x)<=3 else ('0' if '·' in x else ('1' if '公司' in x else '')))
df_银行格式['币种代码'] = df_流水币种['代码编号']
df_银行格式['交易时间'] = all_流水完整['交易日期'].str.extract(r'(\d{1,2}):\d{1,2}:\d{1,2}')+all_流水完整['交易日期'].str.extract(r'\d{1,2}:(\d{1,2}):\d{1,2}')+all_流水完整['交易日期'].str.extract(r'\d{1,2}:\d{1,2}:(\d{1,2})')
df_银行格式['交易时间'] = pd.to_numeric(df_银行格式['交易时间'],errors='coerce').astype(str)+'0'
# df_银行格式['交易金额'] = all_流水完整['交易金额'].astype(float).round(2).astype(str)

df_银行格式['交易金额'] = pd.to_numeric(all_流水完整['交易金额'],errors='coerce').round(2).astype(str)
df_银行格式['折人民币金额'] = df_银行格式['交易金额']
df_银行格式['借贷标志'] = all_流水完整['收付标志'].map(lambda x:'C' if x=='进' else 'D')
df_银行格式['交易对手客户账号'] = all_流水完整['对手卡号']
df_银行格式['交易对手客户名称'] = all_流水完整['对手户名']
df_银行格式['交易流水号'] = all_流水完整['交易流水号']
df_银行格式['交易柜员号'] = all_流水完整['交易柜员号']
df_银行格式['交易机构名称'] = all_流水完整['交易网点名称']
df_银行格式['摘要描述'] = all_流水完整['摘要说明']
df_银行格式['资金用途'] = all_流水完整['渠道']
df_银行格式['终端信息'] = all_流水完整['IP地址']
df_银行格式['账户余额'] = all_流水完整['交易余额']
df_银行格式['备注1'] = all_流水完整['备注']
df_银行格式['反洗钱报告编号'] = '20220306-01'
df_银行格式['场景类型'] = '场景一'

df_银行格式 = df_银行格式.applymap(lambda x: re.sub('nan','',str(x)))
df_银行格式['客户编号'][df_银行格式['客户编号']=='']=df_银行格式['客户账号'][df_银行格式['客户编号']==''] # 用账号填充卡号空白
print(df_银行格式.head(20))


df_tran = df_银行格式
df_tran = df_tran.applymap(lambda x: re.sub('nan',' ',str(x)) )
df_tran[0] = df_tran.apply(lambda x: '|@|'.join(x), axis=1)
pd.DataFrame(df_tran[0]+'|@|').to_csv('C:\\Users\\zqy\\Desktop\\'+'P20210257_JS_FXQ_TRAN_20220306_0001.dat',encoding = 'utf-8',header = None, index=False)
print(df_tran[0])


df_3号令 = pd.DataFrame(columns = ['客户编号','反洗钱报告编号','反洗钱可疑交易笔数','金融机构网点名称','金融机构网点所在地区行政区划代码','反洗钱报告对公对私代码','可疑交易特征描述','反洗钱业务规则名称','反洗钱报告最大交易日期','反洗钱报告最小交易日期','反洗钱报告涉及人民币交易金额','反洗钱报告涉及人民币交易笔数','反洗钱传统渠道人民币交易金额','反洗钱电子银行渠道人民币交易金额','反洗钱新兴电子渠道人民币交易金额','反洗钱其他渠道人民币交易金额','反洗钱传统渠道交易笔数','反洗钱电子银行渠道交易笔数','反洗钱新兴电子渠道交易笔数','反洗钱其他渠道交易笔数','可疑交易措施描述','可疑交易报告报送方向代码','反洗钱可疑行为描述','#报告报送状态代码','报送日期','可疑交易报送标志','#批量导入日期','反洗钱交易主体账户个数','反洗钱中心机构编号','反洗钱中心机构中文名称','#备注二(洗钱类型)','报送次数','接续报告参考标志(0-否 1-是)','非可疑行为描述','可疑交易报告疑点分析说明','可疑交易报告疑点分析及排除理由','反洗钱可疑行为描述2','可疑交易报告疑点分析说明2','风险等级'])
df_3号令['客户编号'] = df_tran.groupby(['客户编号']).size().reset_index()['客户编号']
df_3号令['反洗钱报告对公对私代码'] = df_tran.groupby(['客户编号','对公对私标志']).size().reset_index()['对公对私标志']
df_3号令['#报告报送状态代码'] = '06'
df_3号令 = df_3号令.applymap(lambda x: re.sub('nan',' ',str(x)) )
df_3号令[0] = df_3号令.apply(lambda x: '|@|'.join(x), axis=1)
pd.DataFrame(df_3号令[0]+'|@|').to_csv('C:\\Users\\zqy\\Desktop\\'+'P20210257_JS_FXQ_AML_SSPCS_RPT_20220306_0001.dat',encoding = 'utf-8',header = None, index=False)
print(df_3号令[0])

0
1
交易账卡号       object
交易户名        object
交易账号        object
交易证件号码      object
交易日期        object
交易金额        object
交易余额        object
收付标志        object
对手账号        object
对手卡号        object
现金标志        object
对手户名        object
对手身份证号      object
对手开户银行      object
摘要说明        object
交易币种        object
交易网点名称      object
交易场所        object
交易发生地       object
交易是否成功      object
传票号         object
IP地址        object
MAC地址       object
对手交易余额      object
交易流水号       object
对手余额        object
渠道          object
日志号         object
凭证种类        object
凭证号         object
交易柜员号       object
备注          object
查询反馈结果      object
查询反馈结果原因    object
dtype: object
Empty DataFrame
Columns: [交易账卡号, 交易户名, 交易账号, 交易证件号码, 交易日期, 交易金额, 交易余额, 收付标志, 对手账号, 对手卡号, 现金标志, 对手户名, 对手身份证号, 对手开户银行, 摘要说明, 交易币种, 交易网点名称, 交易场所, 交易发生地, 交易是否成功, 传票号, IP地址, MAC地址, 对手交易余额, 交易流水号, 对手余额, 渠道, 日志号, 凭证种类, 凭证号, 交易柜员号, 备注, 查询反馈结果, 查询反馈结果原因, 值域编号, 值域名称, 代码编号, 代码名称, 英文码]
Index: []

[0 rows x 39 columns]


ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series